In [ ]:
def vis_stock(stock, print_to_folder='__Stock_Analysis/', name_prefix=''):
  try:
    fontsize_common=14
    fontfamily_common='monospace'

    stocks[stock].limit_dataframes('2015-12-31',pricedatemax)
    stocks[stock].cagr_price('2015-12-31',pricedatemax)
    stocks[stock].cagr_price_div('2015-12-31',pricedatemax)

    #Merging price and fundammental data
    for_plot=stocks[stock].hist_price_div.merge(stocks[stock].fundamentals, left_index=True, right_index=True, how='outer')
    #Interest coverage calc
    for_plot['Ones']=1
    for_plot['interestCoverage']=pd.to_numeric(for_plot['interestCoverage'], errors='coerce')
    for_plot['interest/EBIT']=for_plot['Ones'].div(for_plot['interestCoverage']).fillna(0)
    for_plot['interest/EBIT']=pd.to_numeric(for_plot['interest/EBIT'], errors='coerce')
    for_plot.loc[for_plot['interest/EBIT'] == 0,'interest/EBIT'] = np.nan

    for_plot['revenuePerShareInt']=for_plot['revenuePerShare'].interpolate(method='linear', limit_direction = 'backward')
    for_plot['freeCashFlowPerShare']=for_plot['freeCashFlowPerShare'].interpolate(method='linear', limit_direction = 'backward')
    for_plot['netIncomePerShare']=for_plot['netIncomePerShare'].interpolate(method='linear', limit_direction = 'backward')
    for_plot['netProfitMargin']=for_plot['netProfitMargin'].interpolate(method='linear', limit_direction = 'backward')
    for_plot['operatingProfitMargin']=for_plot['operatingProfitMargin'].interpolate(method='linear', limit_direction = 'backward')
    for_plot['interest/EBIT']=for_plot['interest/EBIT'].interpolate(method='linear', limit_direction = 'backward')
    for_plot['netDebtToEBITDA']=for_plot['netDebtToEBITDA'].interpolate(method='linear', limit_direction = 'backward')
    for_plot['earningsYield']=for_plot['earningsYield'].interpolate(method='linear', limit_direction = 'backward')
    for_plot['dividendYield']=for_plot['dividendYield'].interpolate(method='linear', limit_direction = 'backward')
    for_plot['returnOnEquity']=for_plot['returnOnEquity'].interpolate(method='linear', limit_direction = 'backward')

    #Condition
    for_plot['payoutRatio'] = for_plot['payoutRatio'].clip(upper=1)
    for_plot['payoutRatio']=for_plot['payoutRatio'].interpolate(method='linear', limit_direction = 'backward')

    #Plotting TTM data
    ttm=pd.DataFrame(stocks[stock].get_ttm_from_yahoo())
    ttm['date']=pd.to_datetime(ttm['date'])
    ttm['YE']=1/ttm['PEttm']
    ttm.set_index('date', inplace=True)

    fig,ax = plt.subplots(3, 2, figsize=(30,3*5))
    #--------------
    #Plot [0,0]----
    #--------------
    for_plot.plot( 
                y=['close'],
                label=['Close Price'],                                                       
                color = ['dimgrey'],
                linewidth = 3,
                alpha=0.6,
                fontsize=fontsize_common, ax=ax[0,0])
    ax[0,0].legend(fontsize=fontsize_common-2)
    ax[0,0].legend(loc = 'upper left', ncol = 1, fontsize=fontsize_common-2)
    ax001 = ax[0,0].twinx() 
    for_plot.plot(
                y=['revenuePerShareInt'],
                label=[''],                                                       
                color = ['skyblue'],
                linestyle='-',       
                linewidth = 8,
                alpha=0.5,
                fontsize=fontsize_common, ax=ax001)
    for_plot.plot(
                y=['revenuePerShare'],
                label=['Revenue per share'],                  
                color = ['cornflowerblue'],
                linestyle='-',       
                linewidth = 8,
                marker="o", markersize=12,
                alpha=0.5,
                fontsize=fontsize_common, ax=ax001)
    ttm.plot(
                y=['RevPerShare'],
                label=['Revenue TTM'],                                                       
                color = ['cornflowerblue'],
                marker="o", markersize=12,
                alpha=0.75,
                fontsize=fontsize_common, ax=ax001)   
    ax001.legend(loc = 'lower left', ncol = 1, fontsize=fontsize_common-2)

    #--------------
    #Plot [1,0]----
    #--------------
    for_plot.plot(
                y=['freeCashFlowPerShare'], 
                label=['FCF'],                                                       
                color = ['orange'],
                linestyle='-',       
                linewidth = 8,
                alpha=0.5,
                fontsize=fontsize_common, ax=ax[1,0])
    for_plot.plot(
                y=['netIncomePerShare'],
                label=['EPS'],                                                       
                color = ['green'],
                linestyle='-',       
                linewidth = 8,
                alpha=0.25,
                fontsize=fontsize_common, ax=ax[1,0])
    ttm.plot(
                y=['EPS'],
                label=['EPS TTM'],                                                       
                color = ['green'],
                marker="o", markersize=12,
                alpha=0.5,
                fontsize=fontsize_common, ax=ax[1,0])    

    ax101 = ax[1,0].twinx()
    for_plot.plot( 
                y=['close'],  
                color = ['grey'],
                legend=False,                 
                linewidth = 3,
                alpha=0.0,
                fontsize=fontsize_common, ax=ax101)
    ax[1,0].legend(loc = 'upper left', ncol = 1 ,fontsize=fontsize_common-2)
    ax101.set_ylim(ax[1,0].get_ylim()[0], ax[1,0].get_ylim()[1])

    #--------------
    #Plot [0,1]----
    #--------------
    for_plot.plot( 
                y=['adjDividend'],
                label=['Dividends'],                                                       
                color = ['forestgreen'],
                marker="^", markersize=15,
                linewidth = 0,
                alpha=0.7,
                fontsize=fontsize_common, ax=ax[0,1])
    ax[0,1].legend(fontsize=fontsize_common-2)
    ax011 = ax[0,1].twinx() 
    for_plot.plot(
                y=['payoutRatio'],
                label=['DivPayout Ratio'],                                     
                color = ['green'],
                linewidth = 8,
                alpha=0.8,
                fontsize=fontsize_common, ax=ax011)
    for_plot.plot(
                y=['close'],  
                color = ['grey'],
                label=[''],                                                       
                linewidth = 0,
                legend=False, 
                alpha=0.0,
                fontsize=fontsize_common, ax=ax011)
    try:
      ax[0,1].set_ylim(for_plot.adjDividend[for_plot.adjDividend>0].min()*.95,for_plot.adjDividend[for_plot.adjDividend>0].max()*1.05)
      ax011.set_ylim(0, round(for_plot["payoutRatio"].max(),2)+.1)
    except:
      ax[0,1].set_ylim(0,1)
      ax011.set_ylim(0,1)
    ax[0,1].legend(loc = 'upper right', ncol = 1 , fontsize=fontsize_common-2)
    ax011.legend(loc = 'lower right', ncol = 1, fontsize=fontsize_common-2)

    #--------------
    #Plot [2,1]----
    #--------------
    for_plot.plot( 
                y=['netDebtToEBITDA'],
                label=['NetDebt/EBITDA'],                    
                color = ['red'],
                linewidth = 8,
                alpha=0.5,
                fontsize=fontsize_common, ax=ax[2,1])
    ax[2,1].legend(fontsize=fontsize_common-2)
    ax211 = ax[2,1].twinx() 
    for_plot.plot(
                y=['interest/EBIT'],  
                color = ['darkorange'],
                linewidth = 8,
                alpha=0.75,
                fontsize=fontsize_common, ax=ax211)
    for_plot.plot(
                y=['close'],
                label=[''],                                                       
                color = ['grey'],
                linewidth = 0,
                alpha=0.0,
                fontsize=fontsize_common, ax=ax211)

    ax[2,1].legend(loc = 'upper right', ncol = 1 ,fontsize=fontsize_common-2,)
    ax211.legend(loc = 'lower right', ncol = 1, fontsize=fontsize_common-2)
    ax[2,1].axhline(3, linewidth = 2, linestyle='--', color='black', zorder=1, alpha=0.4)	
    ax211.set_ylim(0, max(0.5,round(for_plot['interest/EBIT'][for_plot['interest/EBIT'] > 0].max(),1)+0.1))
    ax[2,1].set_ylim(0, 5)

    #ProfitMargins
    #--------------
    #Plot [2,0]----
    #--------------
    for_plot.plot(
                y=['operatingProfitMargin','netProfitMargin'],  
                color = ['skyblue','green'],
                label=['Operating Margin','Net Margin'],  
                kind='area',
                stacked=False,
                linestyle='-',       
                linewidth = 3,
                alpha=0.2,
                fontsize=fontsize_common, ax=ax[2,0])
    
    ax201 = ax[2,0].twinx()  
    for_plot.plot(
                y=['returnOnEquity'],
                label=['ROE'],  
                color = ['violet'],
                linewidth = 5,
                linestyle='--',
                alpha=0.8,
                fontsize=fontsize_common, ax=ax201)  

    for_plot.plot( 
                y=['close'],  
                label=[''],
                color = ['grey'],
                linewidth = 3,
                alpha=0.0,
                fontsize=fontsize_common, ax=ax201)

    ax[2,0].set_ylim(ymin=0)
    ax201.set_ylim(0, round(for_plot['returnOnEquity'].max(),1)+0.1)
    ax[2,0].legend(loc = 'lower right', ncol = 1 ,fontsize=fontsize_common-2)

    #Earning and dividend yields
    #--------------
    #Plot [1,1]----
    #--------------
    for_plot.plot(
                y=['earningsYield', 'dividendYield'],
                label=['Earning Yield','Dividend Yield'],  
                color = ['purple', 'gold'],
                linestyle='-',       
                linewidth = 8,
                alpha=0.8,
                fontsize=fontsize_common, ax=ax[1,1])
    ttm.plot(
                y=['YE', 'DivYield'],
                label=['YE TTM', 'DivYield TTM'],                                                       
                color = ['purple', 'gold'],
                marker="o", markersize=12,
                alpha=1,
                fontsize=fontsize_common, ax=ax[1,1])    
    ax111 = ax[1,1].twinx()
    for_plot.plot( 
                y=['close'],  
                color = ['grey'],
                linewidth = 3,
                alpha=0.0,
                label=[''],                                                       
                fontsize=fontsize_common, ax=ax111)
    ax[1,1].set_ylim(ymin=0)
    ax111.set_ylim(ax[1,1].get_ylim()[0], ax[1,1].get_ylim()[1])
    ax[1,1].yaxis.set_major_formatter(StrMethodFormatter('{x:,.2f}'))
    ax111.yaxis.set_major_formatter(StrMethodFormatter('{x:,.2f}'))
    ax[1,1].legend(loc = 'upper left', ncol = 1 ,fontsize=fontsize_common-2)
    ax[1,1].axhline(0.05, linewidth = 2, linestyle='--', color='black', zorder=1, alpha=0.4)	

    #Final styling
    for axn in ax.reshape(-1): 
      axn.tick_params(axis='x', labelcolor='black', labelrotation=0, labelbottom=True)
      axn.set_xlabel('')
      plt.setp(axn.xaxis.get_majorticklabels(), ha="center")

    #Final labeling
    ax[0,0].set_title('Share Price & Revenue per Share', fontsize=fontsize_common, family =fontfamily_common, weight='semibold')
    ax[0,0].set_ylabel('Shere price, USD', fontsize=fontsize_common, fontweight='semibold', family =fontfamily_common)
    ax001.set_ylabel('Revenue per share, USD', fontsize=fontsize_common, fontweight='bold', family =fontfamily_common)

    ax[0,1].set_title('Dividends', fontsize=fontsize_common, family =fontfamily_common, weight='semibold')
    ax[0,1].set_ylabel('Dividends, USD', fontsize=fontsize_common, fontweight='bold', family =fontfamily_common)
    ax011.set_ylabel('Dividends payout ratio', fontsize=fontsize_common, fontweight='bold', family =fontfamily_common)

    ax[2,1].set_title('Debt', fontsize=fontsize_common, family =fontfamily_common, weight='semibold')
    ax[2,1].set_ylabel('NetDebt/EBITDA', fontsize=fontsize_common, fontweight='bold', family =fontfamily_common)
    ax211.set_ylabel('Interest/EBIT', fontsize=fontsize_common, fontweight='bold', family =fontfamily_common)

    ax[1,0].set_title('Earnings and Free Cash Flow Per Share', fontsize=fontsize_common, family =fontfamily_common, weight='semibold')
    ax[1,0].set_ylabel('EPS and FCFps, USD', fontsize=fontsize_common, fontweight='bold', family =fontfamily_common)

    ax[2,0].set_title('Effectiveness', fontsize=fontsize_common, family =fontfamily_common, weight='semibold')
    ax[2,0].set_ylabel('Profit margins', fontsize=fontsize_common, fontweight='bold', family =fontfamily_common)
    ax201.set_ylabel('ROE', fontsize=fontsize_common, fontweight='bold', family =fontfamily_common)

    ax[1,1].set_title('Yearning and Dividend Yields', fontsize=fontsize_common, family =fontfamily_common, weight='semibold')
    ax[1,1].set_ylabel('Earnings & div yields', fontsize=fontsize_common, fontweight='bold', family =fontfamily_common)

    #ax001.yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
    #ax001.yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
    
    #Adding stock info
    plt.tight_layout()
    plt.gcf().text(0.03, 1, 'Ticker: ' + stock, fontsize=fontsize_common, style='italic', fontweight='bold')
    plt.gcf().text(0.03, 0.98,\
                   'Sector - ' + stocks[stock].profile.Sector[0] +
                   '. Industry - ' + stocks[stock].profile.Industry[0], 
                   fontsize=fontsize_common, style='italic')    
    plt.gcf().text(0.03, 0.96,\
                   'CAGR: ' + str(round(stocks[stock].cagr_price(),3)) +
                   ', CAGR with divs: ' + str(round(stocks[stock].cagr_price_div(),3)),
                   fontsize=fontsize_common, style='italic')    
    plt.gcf().text(0.03, 0.94, stocks[stock].profile.Description[0][0:285] + '...', fontsize=fontsize_common, style='italic')
    plt.subplots_adjust(top = 0.9, hspace=0.25, wspace = 0.15)

    if name_prefix != '': name_prefix= name_prefix + ' '

    #Export PDF to g-drive
    if print_to_folder != '':
      fig.savefig(stock + '.pdf', quality=90, bbox_inches='tight' )
      shutil.copy(stock + '.pdf' ,'/content/gdrive/My Drive/'+ print_to_folder + name_prefix + stock + '.pdf')


  except:
    pass
